In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [97]:
file_name = "H-2B_Disclosure_Data_FY2019.xlsx"
raw_data = pd.read_excel(file_name, header=0)

raw_data.head(10)

,CASE_NUMBER,DECISION_DATE,VISA_CLASS,SUBMIT_DATE_NEW,CASE_STATUS,CERTIFICATION_BEGIN_DATE,CERTIFICATION_END_DATE,EMPLOYER_NAME,TRADE_NAME_DBA,EMPLOYER_ADDRESS_1,...,WORKSITE_STATE,WORKSITE_POSTAL_CODE,OTHER_WORKSITE_LOCATION,SWA_NAME,JOB_IDNUMBER,JOB_START_DATE,JOB_END_DATE,WORKSITE_ADDRESS_1,WORKSITE_ADDRESS_2,FEIN
0,H-400-18198-951583,2018-10-01,H-2B,2018-08-10 17:42:18.943,Determination Issued - Certification,2018-10-24,2018-11-30,"SIMONS PROPERTY MAINTENANCE, INC.",NaN,3542 Boulden Blvd.,...,UTAH,84093,Y,NaN,NaN,NaT,NaT,2309 EAST 9400 SOUTH,NaN,870581683.0
1,H-400-18199-575646,2018-10-01,H-2B,2018-08-31 11:34:26.506,Determination Issued - Certification,2018-11-14,2019-05-01,"Clay Lowry Forestry, Inc.",NaN,2848 Highway 8 S,...,ARKANSAS,71647,Y,NaN,NaN,NaT,NaT,2848 Hwy. 8 South,NaN,621714069.0
2,H-400-18215-159343,2018-10-01,H-2B,2018-08-17 08:58:25.234,Determination Issued - Partial Certification,2018-11-15,2019-04-01,"Pritchett Forestry Services, Inc.",NaN,198 Old Hwy 10,...,ALABAMA,36782,Y,NaN,NaN,NaT,NaT,198 Old Highway 10,NaN,630973146.0
3,H-400-18215-374903,2018-10-01,H-2B,2018-09-04 09:00:07.433,Determination Issued - Certification,2018-12-01,2019-04-30,"Hamilton Harbor Marina, Inc.",Hamilton Harbor Yacht Club,7065 Hamilton Street,...,FLORIDA,34112,N,NaN,NaN,NaT,NaT,7065 Hamilton Street,NaN,593496970.0
4,H-400-18218-061822,2018-10-01,H-2B,2018-08-07 13:15:23.119,Determination Issued - Certification,2018-11-01,2019-04-01,OR Landscaping LLC,NaN,174 Spur Loop Road,...,WYOMING,83025,Y,NaN,NaN,NaT,NaT,6765 N. Crystal Springs Rd,NaN,453799159.0
5,H-400-18220-156733,2018-10-01,H-2B,2018-08-29 10:06:20.159,Determination Issued - Certification,2018-11-23,2019-04-15,Mount Snow Ltd.,NaN,39 Mount Snow Road,...,VERMONT,5356,N,NaN,NaN,NaT,NaT,39 Mount Snow Rd,NaN,30265116.0
6,H-400-18227-928639,2018-10-01,H-2B,2018-08-29 11:34:05.547,Determination Issued - Certification,2018-11-15,2019-03-29,Contreras Forestry,NaN,870 Greenwood Hwy,...,SOUTH CAROLINA,29138,Y,NaN,NaN,NaT,NaT,870 Greenwood Hwy (report to work),NaN,461689566.0
7,H-400-18232-519453,2018-10-01,H-2B,2018-08-29 11:37:44.007,Determination Issued - Certification,2018-11-15,2019-04-01,Castro Labor Solutions LLC,NaN,237 Red Rock Road,...,GEORGIA,31803,Y,NaN,NaN,NaT,NaT,3782 GA-41 (report to work),NaN,464127338.0
8,H-400-18233-005228,2018-10-01,H-2B,2018-09-01 20:31:42.946,Determination Issued - Certification,2018-12-01,2019-06-30,"Swift Straw II, LLC",NaN,2255 Cumberland Parkway SE,...,GEORGIA,30339,Y,NaN,NaN,NaT,NaT,2255 Cumberland Pkwy SE (report to work),Bldg 1700,452387682.0
9,H-400-18243-789243,2018-10-01,H-2B,2018-08-31 14:08:37.691,Determination Issued - Certification,2018-11-14,2019-07-20,"Total Timberland Services, LLC",NaN,165 Lakemont Dr,...,GEORGIA,30241,Y,NaN,NaN,NaT,NaT,165 Lakemont Dr (report to work),NaN,471097658.0


To start with, let's do some standardization on the class labels, indicated by the column CASE_STATUS.
We will consider "Determination Issued - Certification" as "Certified", and "Determination Issued - Denied" and "Determmination Issued - Rejected" as "Denied", dropping all other records with differing class labels.

In [99]:
df = raw_data.copy()
# Certified as 1, Denied as 0
# df.loc[df["CASE_STATUS"] == "Determination Issued - Certification", "CASE_STATUS"] = "Certified"
# df.loc[df["CASE_STATUS"] == "Determination Issued - Denied", "CASE_STATUS"] = "Denied"
# df.loc[df["CASE_STATUS"] == "Determination Issued - Rejected", "CASE_STATUS"] = "Denied"

df.loc[df["CASE_STATUS"] == "Determination Issued - Certification", "CASE_STATUS"] = 1
df.loc[df["CASE_STATUS"] == "Determination Issued - Denied", "CASE_STATUS"] = 0
df.loc[df["CASE_STATUS"] == "Determination Issued - Rejected", "CASE_STATUS"] = 0

df = df[df["CASE_STATUS"].isin([1, 0])]
df["CASE_STATUS"].value_counts()

1    5873
0     870
Name: CASE_STATUS, dtype: int64

Let's also drop some columns that are irrelevant to our calculations.
In this case, the columns we'll drop are as follows:

Mini play-around, checking to see if this dataset actually works...
Not including other columns that might be relevant but require additional work.

In [145]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, LabelBinarizer
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import tree
from sklearn.metrics import accuracy_score


# Set up the test_df
test_df = df[["CASE_STATUS", "CASE_NUMBER", "AGENT_POC_EMP_REP_BY_AGENT", "SOC_TITLE", "NBR_WORKERS_REQUESTED", "FULL_TIME_POSITION", "NATURE_OF_TEMPORARY_NEED", "BASIC_NUMBER_OF_HOURS", "BASIC_RATE_OF_PAY", "SUPERVISE_OTHER_EMP", "EDUCATION_LEVEL", "TRAINING_REQUIRED", "EMP_EXPERIENCE_REQD", "WORKSITE_STATE"]].copy()

test_df.dropna(inplace=True)
test_df.set_index("CASE_NUMBER", inplace=True)
test_df.drop_duplicates(inplace=True)

test_df['AGENT_POC_EMP_REP_BY_AGENT'] = test_df['AGENT_POC_EMP_REP_BY_AGENT'].map({'Yes': 1, 'No': 0})
test_df['FULL_TIME_POSITION'] = test_df['FULL_TIME_POSITION'].map({'Y': 1, 'N': 0})
test_df["SUPERVISE_OTHER_EMP"] = test_df['SUPERVISE_OTHER_EMP'].map({'Yes': 1, 'No': 0})
test_df["TRAINING_REQUIRED"] = test_df['TRAINING_REQUIRED'].map({'Yes': 1, 'No': 0})
test_df["EMP_EXPERIENCE_REQD"] = test_df['EMP_EXPERIENCE_REQD'].map({'Yes': 1, 'No': 0})

test_df["WORKSITE_STATE"] = test_df["WORKSITE_STATE"].str.upper()

# Isolate labels, features
labels = test_df["CASE_STATUS"].values.ravel()
labels = labels.astype('int')
print(labels.dtype)
features = test_df.drop("CASE_STATUS", axis=1)

# One hot encoding needed for SOC_TITLE (199), NAICS_CODE(?), NATURE_OF_TEMPORARY_NEED (4), EDUCATION_LEVEL (5), WORKSITE_STATE (50)
# Must pass in indices of columns that need to be categorized?

# Two options: 
# 1) LabelEncoder(?) -> OneHotEncoder
# 2) LabelBinarizer (but only works on one column data).

# scaler_1 = OneHotEncoder(ColumnTransformer = [1, 4, 8, 11])
# scaler_2 = StandardScaler()

preprocess = make_column_transformer(
    # TODO: IS THE HANDLE UNKNOWN PARAMETER CORRECT?
    (['SOC_TITLE', 'NATURE_OF_TEMPORARY_NEED', 'EDUCATION_LEVEL', 'WORKSITE_STATE'], OneHotEncoder(handle_unknown='ignore'))
)
tree_clf = tree.DecisionTreeClassifier(criterion="entropy")
pipeline = Pipeline([("preprocess", preprocess), ("tree_clf", tree_clf)])

nested_scores = cross_val_score(pipeline, features, labels, cv=5)
print("Accuracy of model:", nested_scores.mean() * 100)

int32


C:\Users\Matthew\Anaconda3\lib\site-packages\sklearn\compose\_column_transformer.py:751: DeprecationWarning: `make_column_transformer` now expects (transformer, columns) as input tuples instead of (columns, transformer). This has been introduced in v0.20.1. `make_column_transformer` will stop accepting the deprecated (columns, transformer) order in v0.22.
  warnings.warn(message, DeprecationWarning)


Accuracy of model: 86.68780833379219


In [151]:
# But how trust worthy is this accuracy? Remember, we have a heavy class imbalance.
print(5873 / (870 + 5873))

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, classification_report

# Let's get a better idea of how accurate our predictor (using decision trees) actually was.
predicted_labels = cross_val_predict(pipeline, features, labels, cv=5)
print(confusion_matrix(labels, predicted_labels))
print(classification_report(labels, predicted_labels))

0.8709773098027584
[[ 296  477]
 [ 302 4792]]
              precision    recall  f1-score   support

           0       0.49      0.38      0.43       773
           1       0.91      0.94      0.92      5094

    accuracy                           0.87      5867
   macro avg       0.70      0.66      0.68      5867
weighted avg       0.85      0.87      0.86      5867

